In [58]:
import funzioni02 as f2
import funzioni01 as f1
import qiskit.quantum_info as qi
from copy import deepcopy
from sympy import *
import numpy as np
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity, Statevector
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts
plt.rcParams['figure.figsize'] = [15, 10]
%load_ext autoreload
%autoreload 2

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ibmqfactory.load_account:WARNING:2022-04-09 14:23:38,455: Credentials are already in use. The existing account in the session will be replaced.


In [59]:
shots=32000
precision=50
steps=99
check=["4copy_check", [0,2,4,6]]
evo_type="complete_evolution"
time=np.pi

backend_aus=backend_sim_jakarta

id_tomo = "624dbafbf65d78c03a38fbb6"
id_cal_identity = "624dbafe182d0267144c22e8"
id_cal_itself = "624dbb01cfe45c43e6e59cbd"
id_cal_qiskit = "624dbb03aacb9beb455f46e3"

In [60]:
job=backend.retrieve_job(id_tomo)

job_cal_our_identity=backend.retrieve_job(id_cal_identity)

job_cal_our_itself_new=backend.retrieve_job(id_cal_itself)

job_cal=backend.retrieve_job(id_cal_qiskit)

In [61]:
qr_cal = QuantumRegister(7)
qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
cal_circ, state_labels = mc.complete_meas_cal(qubit_list=[1,3,5,0,2,4,6], qr=qr_cal, circlabel='mcal') #[1,3,5,0,2,4]


In [5]:
qcs_na[0].draw()

┌──────────────┐ ░               
 q_0: ──────────────────────┤0             ├─░───────────────
           ┌────┐┌─────────┐│              │ ░ ┌───┐┌─┐      
 q_1: ─────┤0   ├┤0        ├┤4             ├─░─┤ H ├┤M├──────
           │    ││         ││              │ ░ └───┘└╥┘      
 q_2: ─────┤  U ├┤         ├┤1             ├─░───────╫───────
      ┌───┐│    ││         ││              │ ░ ┌───┐ ║ ┌─┐   
 q_3: ┤ X ├┤1   ├┤1 Mdg_dg ├┤5 4copy_check ├─░─┤ H ├─╫─┤M├───
      └───┘└────┘│         ││              │ ░ └───┘ ║ └╥┘   
 q_4: ───────────┤         ├┤2             ├─░───────╫──╫────
      ┌───┐      │         ││              │ ░ ┌───┐ ║  ║ ┌─┐
 q_5: ┤ X ├──────┤2        ├┤6             ├─░─┤ H ├─╫──╫─┤M├
      └───┘      └─────────┘│              │ ░ └───┘ ║  ║ └╥┘
 q_6: ──────────────────────┤3             ├─░───────╫──╫──╫─
                            └──────────────┘ ░       ║  ║  ║ 
c0: 3/═══════════════════════════════════════════════╩══╩══╩═
                                                     0  1  2

In [63]:
qcs_calib_id, _ = f2.calibration_cirquits(type="complete_evolution_remake", n_steps=steps, time=time, q_anc=check[1], check="yes", check_type=check[0])
                                        # complete_evolution, itself, itself_whole

In [64]:
qcs_calib_id[1].draw()

┌──────────────┐      ░          ┌─┐         
q2625_0: ────────────────┤0             ├──────░──────────┤M├─────────
         ┌───┐┌────┐┌───┐│              │┌───┐ ░ ┌─┐      └╥┘         
q2625_1: ┤ X ├┤0   ├┤ X ├┤4             ├┤ X ├─░─┤M├───────╫──────────
         └───┘│    │└───┘│              │└───┘ ░ └╥┘       ║ ┌─┐      
q2625_2: ─────┤    ├─────┤1             ├──────░──╫────────╫─┤M├──────
         ┌───┐│    │┌───┐│              │      ░  ║ ┌─┐    ║ └╥┘      
q2625_3: ┤ X ├┤1 U ├┤ X ├┤5 4copy_check ├──────░──╫─┤M├────╫──╫───────
         └───┘│    │└───┘│              │      ░  ║ └╥┘    ║  ║ ┌─┐   
q2625_4: ─────┤    ├─────┤2             ├──────░──╫──╫─────╫──╫─┤M├───
         ┌───┐│    │┌───┐│              │      ░  ║  ║ ┌─┐ ║  ║ └╥┘   
q2625_5: ┤ X ├┤2   ├┤ X ├┤6             ├──────░──╫──╫─┤M├─╫──╫──╫────
         └───┘└────┘└───┘│              │      ░  ║  ║ └╥┘ ║  ║  ║ ┌─┐
q2625_6: ────────────────┤3             ├──────░──╫──╫──╫──╫──╫──╫─┤M├
                         └──────────────┘      ░  ║  ║  ║  ║  ║  ║ └╥┘
 c580: 7/═════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                                  0  1  2  3  4  5  6

In [8]:
qcs_calib_itself, _ = f2.calibration_cirquits(type="itself_whole", n_steps=steps, time=time, q_anc=check[1], check="yes", check_type=check[0])
 

In [9]:
qcs_calib_itself[1].draw()

┌──────────────┐ ░          ┌─┐         
q425_0: ───────────┤0             ├─░──────────┤M├─────────
        ┌───┐┌────┐│              │ ░ ┌─┐      └╥┘         
q425_1: ┤ X ├┤0   ├┤4             ├─░─┤M├───────╫──────────
        └───┘│    ││              │ ░ └╥┘       ║ ┌─┐      
q425_2: ─────┤    ├┤1             ├─░──╫────────╫─┤M├──────
             │    ││              │ ░  ║ ┌─┐    ║ └╥┘      
q425_3: ─────┤1 U ├┤5 4copy_check ├─░──╫─┤M├────╫──╫───────
             │    ││              │ ░  ║ └╥┘    ║  ║ ┌─┐   
q425_4: ─────┤    ├┤2             ├─░──╫──╫─────╫──╫─┤M├───
             │    ││              │ ░  ║  ║ ┌─┐ ║  ║ └╥┘   
q425_5: ─────┤2   ├┤6             ├─░──╫──╫─┤M├─╫──╫──╫────
             └────┘│              │ ░  ║  ║ └╥┘ ║  ║  ║ ┌─┐
q425_6: ───────────┤3             ├─░──╫──╫──╫──╫──╫──╫─┤M├
                   └──────────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
c287: 7/═══════════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                       0  1  2  3  4  5  6

In [10]:
#job=execute(qcs, backend_aus, shots=shots)

#job_cal_our_identity=execute(qcs_calib, backend = backend_aus, shots=shots)

#job_cal_our_itself_new=execute(qcs_calib_itself, backend = backend_aus, shots=shots)

#job_cal=execute(cal_circ, backend_aus, shots=shots)

In [212]:
#building the circuit matrix for the calibration itself

qr_a = QuantumRegister(7)
qc_ausiliare = QuantumCircuit(qr_a)

qc_ausiliare2=qcs_calib_itself[0].copy()
qc_ausiliare2.remove_final_measurements()

qc_ausiliare.append(qc_ausiliare2,[qr_a[3], qr_a[0], qr_a[4], qr_a[1],  qr_a[5],  qr_a[2], qr_a[6]])#[1,3,5,0,2,4,6])#[0,1,2,3,4,5,6])

qc_ausiliare2.draw()

┌──────────────┐
q422_0: ──────┤0             ├
        ┌────┐│              │
q422_1: ┤0   ├┤4             ├
        │    ││              │
q422_2: ┤    ├┤1             ├
        │    ││              │
q422_3: ┤1 U ├┤5 4copy_check ├
        │    ││              │
q422_4: ┤    ├┤2             ├
        │    ││              │
q422_5: ┤2   ├┤6             ├
        └────┘│              │
q422_6: ──────┤3             ├
              └──────────────┘

In [213]:
qc_ausiliare.draw()

┌──────────────┐
q4912_0: ┤1             ├
         │              │
q4912_1: ┤3             ├
         │              │
q4912_2: ┤5             ├
         │              │
q4912_3: ┤0 cal_0000000 ├
         │              │
q4912_4: ┤2             ├
         │              │
q4912_5: ┤4             ├
         │              │
q4912_6: ┤6             ├
         └──────────────┘

In [188]:
cirquit_itself_matrix = f2.matrix_from_cirquit(qc_ausiliare, type="numpy")
#Matrix(cirquit_itself_matrix)

In [189]:
meas_fitter_itself = mc.CompleteMeasFitter(job_cal_our_itself_new.result(), state_labels=state_labels)
meas_fitter_identity = mc.CompleteMeasFitter(job_cal_our_identity.result(), state_labels=state_labels)
meas_fitter = mc.CompleteMeasFitter(job_cal.result(), state_labels=state_labels)

U_tilde_itself = meas_fitter_itself.cal_matrix
U_tilde_identity = meas_fitter_identity.cal_matrix
U_tilde_qiskit = meas_fitter.cal_matrix

In [190]:
U_tilde_itself

array([[1.096875e-02, 3.784375e-02, 2.759375e-02, ..., 5.000000e-04,
        1.250000e-04, 2.500000e-04],
       [4.875000e-03, 8.625000e-03, 1.096250e-01, ..., 2.000000e-03,
        3.750000e-04, 1.281250e-03],
       [9.687500e-04, 3.043750e-02, 1.312500e-03, ..., 3.125000e-05,
        3.375000e-03, 1.031250e-03],
       ...,
       [2.500000e-03, 1.750000e-03, 1.531250e-03, ..., 1.562500e-04,
        9.375000e-05, 5.312500e-04],
       [8.125000e-04, 2.812500e-04, 9.375000e-04, ..., 9.375000e-05,
        3.125000e-05, 0.000000e+00],
       [1.153125e-02, 6.812500e-03, 4.750000e-03, ..., 1.250000e-04,
        3.125000e-05, 0.000000e+00]])

In [191]:
U_tilde_qiskit

array([[7.2765625e-01, 1.9312500e-02, 3.1406250e-02, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0146875e-01, 8.1984375e-01, 4.3125000e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.1250000e-03, 1.2500000e-04, 7.1812500e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 7.2906250e-01,
        4.7812500e-03, 3.2812500e-02],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.3750000e-05,
        6.1331250e-01, 1.5687500e-02],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.7187500e-03,
        9.8281250e-02, 6.9878125e-01]])

In [214]:
qr_basi = QuantumRegister(7)
qc_basi = QuantumCircuit(qr_basi)

#qc_basi.swap(5,4)
#qc_basi.swap(5,2)
#qc_basi.swap(1,3)
#qc_basi.swap(0,1)

qcs_basis = state_tomography_circuits(qc_basi,[0,1,2])


C_matrices_identity = []
C_matrices_itself = []

C_itself = np.dot(U_tilde_itself, np.linalg.inv(cirquit_itself_matrix))
C_identity = U_tilde_identity

for base_cirquit in qcs_basis:
    
    base_cirquit.remove_final_measurements()

    qca_matrix = f2.matrix_from_cirquit(base_cirquit, type="numpy")
    qca_matrixH = qca_matrix.conj().T

    C_aus_itself = np.linalg.multi_dot([U_tilde_qiskit, np.asarray(qca_matrix), np.linalg.inv(U_tilde_qiskit), U_tilde_itself, np.linalg.inv(cirquit_itself_matrix),  qca_matrixH])
    C_aus_identity = np.dot(np.asarray(qca_matrix),C_identity).dot(qca_matrixH)

    C_matrices_itself.append(np.asarray(C_aus_itself))
    C_matrices_identity.append(np.asarray(C_aus_identity))

In [215]:
from copy import deepcopy
meas_fitter_ours_id = []
meas_fitter_ours_itself = []

for C_new in C_matrices_itself:
    meas_fitter_our_aus = deepcopy(meas_fitter_itself)
    meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

    meas_fitter_ours_itself.append(meas_fitter_our_aus)

for C_new in C_matrices_identity:
    meas_fitter_our_aus = deepcopy(meas_fitter_identity)
    meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

    meas_fitter_ours_id.append(meas_fitter_our_aus)



In [216]:
np.shape(meas_fitter.cal_matrix)

(128, 128)

In [217]:
U = f2.Trotter_N_approx(steps=steps, tempo=time, precision=precision)

U2=[
    [U[3*8+3],U[3*8+5],U[3*8+6],0],
    [U[5*8+3],U[5*8+5],U[5*8+6],0],
    [U[6*8+3],U[6*8+5],U[6*8+6],0],
    [0,0,0,1]
]

qc=QuantumCircuit(2, name="U, t=%.2f pi"%(time/np.pi))
qc.unitary(U2,[0,1])    
trans_qc=transpile(qc,basis_gates=['cx','x','sx','rz']) 

qr2=QuantumRegister(3, name="q")
qc2=QuantumCircuit(qr2)

#qc2.x([qr2[1],qr2[2]])
qc2.append(trans_qc,[qr2[0],qr2[1]])
qc2.append(f2.Mdg_circquit().inverse(),[qr2[0],qr2[1],qr2[2]])

target_state = Statevector.from_instruction(qc2)

In [218]:
#qcs2=
qc2.draw()

┌───────────────┐┌─────────┐
q_0: ┤0              ├┤0        ├
     │  U, t=1.00 pi ││         │
q_1: ┤1              ├┤1 Mdg_dg ├
     └───────────────┘│         │
q_2: ─────────────────┤2        ├
                      └─────────┘

In [219]:
f2.matrix_from_cirquit(qc2)

Matrix([
[                            0,                             0,                             0, 1.0,                             0,                             0,                             0,   0],
[0.0001020679 - 0.0021063946*I,  -9.87301e-5 + 0.0010531973*I, 0.9999966622 + 0.0010531973*I,   0,                             0,                             0,                             0,   0],
[ -9.87301e-5 + 0.0010531973*I, 0.9999966622 - 0.0021063946*I, 0.0001020679 + 0.0010531973*I,   0,                             0,                             0,                             0,   0],
[                            0,                             0,                             0,   0, 0.9999966622 + 0.0010531973*I, 0.0001020679 + 0.0010531973*I,  -9.87301e-5 - 0.0021063946*I,   0],
[0.9999966622 + 0.0010531973*I, 0.0001020679 + 0.0010531973*I,  -9.87301e-5 - 0.0021063946*I,   0,                             0,                             0,                             0,   0],
[

In [220]:
#target_state = (np.asarray(target_state))
#target_state

In [221]:
target_state=((One^One^Zero).to_matrix())


In [222]:
reps=1

#target_state = (One^One^Zero).to_matrix()

fids=np.zeros([reps,5])
#fids_mean=np.zeros(4)
#fids_dev=np.zeros(4)

for j in range(reps):
    
    #res = jobs_evo_result[j]
    res=job.result()

    print(j)
    
    new_res, new_res_nm = f2.mitigate(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter)
    new_res_our, new_res_nm = f2.mitigate(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_ours_id)
    new_res_our_itself, new_res_nm = f2.mitigate(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_ours_itself)
    new_res_not_mitigated = f2.mitigate(res, ancillas_conditions=f2.bin_list(4))

    new_res_our_itself.get_counts(-1)

    fids[j,0] = f2.fidelity_count(new_res_not_mitigated, qcs_na, target_state)
    fids[j,1] = f2.fidelity_count(new_res_nm, qcs_na, target_state)
    fids[j,2] = f2.fidelity_count(new_res, qcs_na, target_state)
    fids[j,3] = f2.fidelity_count(new_res_our, qcs_na, target_state)
    fids[j,4] = f2.fidelity_count(new_res_our_itself, qcs_na, target_state)

#for i in range(4):
#    fids_mean[i]=np.mean(fids[:,i])
#    fids_dev[i]=np.std(fids[:,i])


0
0
tens_fitt
1
tens_fitt
2
tens_fitt
3
tens_fitt
4
tens_fitt
5
tens_fitt
6
tens_fitt
7
tens_fitt
8
tens_fitt
9
tens_fitt
10
tens_fitt
11
tens_fitt
12
tens_fitt
13
tens_fitt
14
tens_fitt
15
tens_fitt
16
tens_fitt
17
tens_fitt
18
tens_fitt
19
tens_fitt
20
tens_fitt
21
tens_fitt
22
tens_fitt
23
tens_fitt
24
tens_fitt
25
tens_fitt
26
tens_fitt
0
0 (128, 128)
deepcopy
1
1 (128, 128)
deepcopy
2
2 (128, 128)
deepcopy
3
3 (128, 128)
deepcopy
4
4 (128, 128)
deepcopy
5
5 (128, 128)
deepcopy
6
6 (128, 128)
deepcopy
7
7 (128, 128)
deepcopy
8
8 (128, 128)
deepcopy
9
9 (128, 128)
deepcopy
10
10 (128, 128)
deepcopy
11
11 (128, 128)
deepcopy
12
12 (128, 128)
deepcopy
13
13 (128, 128)
deepcopy
14
14 (128, 128)
deepcopy
15
15 (128, 128)
deepcopy
16
16 (128, 128)
deepcopy
17
17 (128, 128)
deepcopy
18
18 (128, 128)
deepcopy
19
19 (128, 128)
deepcopy
20
20 (128, 128)
deepcopy
21
21 (128, 128)
deepcopy
22
22 (128, 128)
deepcopy
23
23 (128, 128)
deepcopy
24
24 (128, 128)
deepcopy
25
25 (128, 128)
deepcopy
2

In [223]:
fids

array([[0.67266522, 0.74029464, 0.92706888, 0.98472036, 0.99089243]])

In [ ]:
def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    
    # Sum interactions
    H = XXs + YYs + ZZs
    
    # Return Hamiltonian
    return H

def U_heis3(t):
    # Compute XXX Hamiltonian for 3 spins in a line
    H = H_heis3()
    
    # Return the exponential of -i multipled by time t multipled by the 3 spin XXX Heisenberg Hamilonian 
    return (t * H).exp_i()

In [ ]:
initial_state = One^One^Zero
target_state2 = U_heis3(float(time)) @ initial_state #Zero^One^One

np.abs((~target_state2 @ U_heis3(float(time)) @ initial_state).eval())**2

1.0000000000000009

$\hbar$